In [1]:
import numpy as np
import os
import sys
import copy

import torch
torch.set_num_threads=4
from torch import nn
from torchsummary import summary

import importlib
import pickle as pk

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as anm

sys.path.insert(0, "../src/")
import data
import model
import train

#import nmrglue as ng
import scipy
import scipy.io

np.random.seed(123)

In [2]:
mod = "Ensemble_PIPNet_2022_03_01_noise_005"

in_dir = f"../data/{mod}/"
par_dir = f"../figures/{mod}/"
fig_dir = f"../figures/test_2D/"

exp_dir = f"../data/experimental_spectra/2D/"

In [3]:
if not os.path.exists(in_dir):
    raise ValueError(f"Unknown model: {mod}")
    
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)

In [4]:
with open(f"{par_dir}params.pk", "rb") as F:
    model_pars, loss_pars, data_pars = pk.load(F)

# Load model

In [5]:
# Load loss and learning rate
all_lrs = np.load(in_dir + "all_lrs.npy")
all_losses = np.load(in_dir + "all_losses.npy")
all_val_losses = np.load(in_dir + "all_val_losses.npy")

mean_losses = np.mean(all_losses, axis=1)
mean_val_losses = np.mean(all_val_losses, axis=1)

n_chk = all_losses.shape[0]
best_chk = np.argmin(mean_val_losses)
print(best_chk)

net = model.ConvLSTMEnsemble(**model_pars)
net.load_state_dict(torch.load(in_dir + f"checkpoint_{best_chk+1}_network", map_location=torch.device("cpu")))
net = net.eval()

118


In [6]:
def make_input(X, ws, data_pars, Xi=None, x_max=0.25):
    
    inds = np.argsort(ws)
    X_torch = torch.Tensor(X[:, inds])
    X_torch = torch.unsqueeze(X_torch, dim=2)
    
    M = torch.max(X_torch)
    
    X_torch /= M
    X_torch *= x_max
    
    if data_pars["encode_w"]:
        W = torch.Tensor(ws[inds])
        W = torch.unsqueeze(W, dim=0)
        W = torch.unsqueeze(W, dim=2)
        W = torch.unsqueeze(W, dim=3)
        W = W.repeat(X_torch.shape[0], 1, 1, X_torch.shape[-1])
        
        if data_pars["norm_wr"]:
            W -= data_pars["mas_w_range"][0]
            W /= data_pars["mas_w_range"][1] - data_pars["mas_w_range"][0]
    
    if Xi is not None:
        X_imag = torch.Tensor(Xi[inds])
        X_imag = torch.unsqueeze(X_imag, dim=0)
        X_imag = torch.unsqueeze(X_imag, dim=2)
        X_imag /= M
        X_imag *= x_max
        
        X_torch = torch.cat([X_torch, X_imag, W], dim=2)
    
    else:
        X_torch = torch.cat([X_torch, W], dim=2)
    
    return X_torch, ws[inds]

In [8]:
for sens in [1., 2., 5., 10., 20., 50., 100.]:

    for f in os.listdir(exp_dir):
        if f.endswith(".mat"):
            print(f)
            m = scipy.io.loadmat(exp_dir + f)

            X = m["newsignal2"] / np.sum(m["newsignal2"], axis=(0, 1))[None, None, :]
            X /= np.max(X, axis=(0,1))[None, None, :] * sens

            ws = m["wr"][0].astype(float)

            X_torch, sel_ws = make_input(X.transpose([0, 2, 1]), ws, data_pars, x_max=0.25*sens)

            with torch.no_grad():
                y_pred, y_err, ys = net(X_torch)

            fig = plt.figure(figsize=(4,3))
            ax = fig.add_subplot(1,1,1)
            ax.contour(X[:, :, 0], cmap="Blues", linewidths=0.5, vmin=0., vmax=0.1)
            ax.contour(y_pred[:, -1, :].numpy(), cmap="Reds", linewidths=0.5, vmin=0, vmax=0.01)

            fig.tight_layout()
            plt.savefig(f"{fig_dir}preds_{f.split('_')[0]}_sens_{sens:.0f}.pdf")
            plt.close()
            
            np.savetxt(f"{fig_dir}preds_{f.split('_')[0]}_sens_{sens:.0f}_pred.csv", y_pred[:, -1, :], fmt="%.4e", delimiter=",")
            np.savetxt(f"{fig_dir}preds_{f.split('_')[0]}_sens_{sens:.0f}_errs.csv", y_err[:, -1, :], fmt="%.4e", delimiter=",")
            

tyrosine_unshear_gls_220302_0toInf_0to5.mat
ampicillin_unshear_gandl_220307.mat
tyrosine_unshear_gls_220302_0toInf_0to5.mat
ampicillin_unshear_gandl_220307.mat
tyrosine_unshear_gls_220302_0toInf_0to5.mat
ampicillin_unshear_gandl_220307.mat
tyrosine_unshear_gls_220302_0toInf_0to5.mat
ampicillin_unshear_gandl_220307.mat
tyrosine_unshear_gls_220302_0toInf_0to5.mat
ampicillin_unshear_gandl_220307.mat
tyrosine_unshear_gls_220302_0toInf_0to5.mat
ampicillin_unshear_gandl_220307.mat
tyrosine_unshear_gls_220302_0toInf_0to5.mat
ampicillin_unshear_gandl_220307.mat
